### Visualize (Hand Coupling / Save Video)

In [1]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
from util import rpy2R,r2quat,get_uv_dict_nc,get_p_target_nc
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


### Parse `scene_common_rig_psyonic.xml`

In [2]:
xml_path = '../asset/common_rig/scene_common_rig_psyonic_robot.xml'
env = MuJoCoParserClass(name='Common-Rig',rel_xml_path=xml_path,VERBOSE=True)
# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    body_idx = env.body_names.index(body_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==body_idx]
    for geom_idx in geom_idxs:
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,0.5]
print ("Done.")

dt:[0.0050] HZ:[200]
n_dof (=nv):[61]
n_geom:[65]
geom_names:['floor', 'base', 'root2spine', 'spine2neck', 'neck2rclavicle', 'neck2lclavicle', 'neck2head', 'rclavicle2rshoulder', 'rshoulder', 'rshoulder2relbow', 'relbow2rwrist', 'rwrist2rmiddle', 'rmiddle2rthumb', 'rmiddle2rindex', 'rmiddle_l01', 'rmiddle2rring', 'rmiddle2rpinky', 'rthumb_l12', 'rthumb_l2end', 'rindex_l01', 'rindex_l12', 'rindex_l2end', 'rmiddle_l12', 'rmiddle_l2end', 'rring_l01', 'rring_l12', 'rring_l2end', 'rpinky_l01', 'rpinky_l12', 'rpinky_l2end', 'lclavicle2lshoulder', 'lshoulder', 'lshoulder2lelbow', 'lelbow2lwrist', 'lwrist2lthumb', 'lmiddle2lthumb', 'lmiddle2rindex', 'lmiddle_l01', 'lmiddle2rring', 'lmiddle2lpinky', 'lthumb_l12', 'lthumb_l2end', 'lindex_l01', 'lindex_l12', 'lindex_l2end', 'lmiddle_l12', 'lmiddle_l2end', 'lring_l01', 'lring_l12', 'lring_l2end', 'lpinky_l01', 'lpinky_l12', 'lpinky_l2end', 'head', 'nose', 'base2rpelvis', 'rpelvis2rknee', 'rknee2rankle', 'rankle', 'rfoot', 'base2lpelvis', 'lpelvis2

In [3]:
for (i,n) in enumerate (env.joint_names):
    print ("[%d] %s"%(i,n))

[0] base
[1] root1
[2] root2
[3] root3
[4] spine
[5] spinex
[6] spinez
[7] rc
[8] rs1
[9] rs2
[10] re
[11] rw1
[12] rw2
[13] rthumb_q1
[14] rthumb_q2
[15] rindex_q1
[16] rindex_q2
[17] rmiddle_q1
[18] rmiddle_q2
[19] rring_q1
[20] rring_q2
[21] rpinky_q1
[22] rpinky_q2
[23] lc
[24] ls1
[25] ls2
[26] le
[27] lw1
[28] lw2
[29] lthumb_q1
[30] lthumb_q2
[31] lindex_q1
[32] lindex_q2
[33] lmiddle_q1
[34] lmiddle_q2
[35] lring_q1
[36] lring_q2
[37] lpinky_q1
[38] lpinky_q2
[39] head1
[40] head2
[41] head3
[42] rp1
[43] rp2
[44] rp3
[45] rk
[46] ra1
[47] ra2
[48] ra3
[49] lp1
[50] lp2
[51] lp3
[52] lk
[53] la1
[54] la2
[55] la3


In [4]:
# import pickle

# data_number  = "22_01"
# pkl_path = "../data/pkl/VAAI_DIRECT_" + data_number + "_a_M1.pkl"

# with open(pkl_path,'rb') as f:
#     data = pickle.load(f)

# L = data['length']

# q_full = np.zeros((L,62))

# q_full[:,:7] = data['qpos'][:,:7]

# # Right Arm
# # q_full[:,14] = - data['qpos'][:,16] - np.radians(90)
# q_full[:,14] = - data['qpos'][:,16] - np.radians(110)
# q_full[:,15] = data['qpos'][:,14]
# q_full[:,16] = data['qpos'][:,17] - np.radians(90)
# # q_full[:,17] = data['qpos'][:,18]
# q_full[:,17] = data['qpos'][:,18] + np.radians(30)
# q_full[:,18] = data['qpos'][:,20]

# # Right Hand
# q_full[:,19] = -data['qpos'][:,21]
# q_full[:,20] = -(data['qpos'][:,22]+data['qpos'][:,23])/2
# q_full[:,21] = (data['qpos'][:,24]+data['qpos'][:,25]+data['qpos'][:,26])/3
# q_full[:,23] = (data['qpos'][:,27]+data['qpos'][:,28]+data['qpos'][:,29])/3
# q_full[:,25] = (data['qpos'][:,30]+data['qpos'][:,31]+data['qpos'][:,32])/3
# q_full[:,27] = (data['qpos'][:,33]+data['qpos'][:,34]+data['qpos'][:,35])/3

# q_full[:,22] = 1.05851325*q_full[:,21]+0.72349796
# q_full[:,24] = 1.05851325*q_full[:,23]+0.72349796
# q_full[:,26] = 1.05851325*q_full[:,25]+0.72349796
# q_full[:,28] = 1.05851325*q_full[:,27]+0.72349796

# # Left Arm
# # q_full[:,30] = data['qpos'][:,39] + np.radians(90)
# q_full[:,30] = data['qpos'][:,39] + np.radians(110)
# q_full[:,31] = -data['qpos'][:,37]
# q_full[:,32] = -data['qpos'][:,40] - np.radians(90)
# # q_full[:,33] = data['qpos'][:,41]
# q_full[:,33] = data['qpos'][:,41] - np.radians(30)
# q_full[:,34] = -data['qpos'][:,43]

# # Left Hand
# q_full[:,35] = -data['qpos'][:,44]
# q_full[:,36] = (data['qpos'][:,45]+data['qpos'][:,46])/2
# q_full[:,37] = -(data['qpos'][:,47]+data['qpos'][:,48]+data['qpos'][:,49])/3
# q_full[:,39] = -(data['qpos'][:,50]+data['qpos'][:,51]+data['qpos'][:,52])/3
# q_full[:,41] = -(data['qpos'][:,53]+data['qpos'][:,54]+data['qpos'][:,55])/3
# q_full[:,43] = -(data['qpos'][:,56]+data['qpos'][:,57]+data['qpos'][:,58])/3

# q_full[:,38] = 1.05851325*q_full[:,37]+0.72349796
# q_full[:,40] = 1.05851325*q_full[:,39]+0.72349796
# q_full[:,42] = 1.05851325*q_full[:,41]+0.72349796
# q_full[:,44] = 1.05851325*q_full[:,43]+0.72349796

In [5]:
import pickle

data_number  = "22_01"
pkl_path = "../data/pkl/VAAI_DIRECT_" + data_number + "_a_M1.pkl"

with open(pkl_path,'rb') as f:
    data = pickle.load(f)

inpainted_data = np.load("../data/" + data_number[:2] + "a_inpaint.npy")

L = inpainted_data.shape[0]
q_full = np.zeros((L,62))
q_full[:,:7] = data['qpos'][:,:7]

# Right Arm
q_full[:,14] = - data['qpos'][:,16] - np.radians(70)
q_full[:,15] = data['qpos'][:,14]
q_full[:,16] = data['qpos'][:,17] - np.radians(90)
q_full[:,17] = data['qpos'][:,18] + np.radians(30)
q_full[:,18] = data['qpos'][:,20]

# Right Hand
q_full[:,19] = -inpainted_data[:,21]
q_full[:,20] = -(inpainted_data[:,22]+inpainted_data[:,23])/2
q_full[:,21] = (inpainted_data[:,24]+inpainted_data[:,25]+inpainted_data[:,26])/3
q_full[:,23] = (inpainted_data[:,27]+inpainted_data[:,28]+inpainted_data[:,29])/3
q_full[:,25] = (inpainted_data[:,30]+inpainted_data[:,31]+inpainted_data[:,32])/3
q_full[:,27] = (inpainted_data[:,33]+inpainted_data[:,34]+inpainted_data[:,35])/3

q_full[:,22] = 1.05851325*q_full[:,21]+0.72349796
q_full[:,24] = 1.05851325*q_full[:,23]+0.72349796
q_full[:,26] = 1.05851325*q_full[:,25]+0.72349796
q_full[:,28] = 1.05851325*q_full[:,27]+0.72349796

# Left Arm
q_full[:,30] = data['qpos'][:,39] + np.radians(90)
q_full[:,31] = -data['qpos'][:,37]
q_full[:,32] = -data['qpos'][:,40] - np.radians(90)
q_full[:,33] = data['qpos'][:,41]
q_full[:,34] = -data['qpos'][:,43]

# Left Hand
q_full[:,35] = -inpainted_data[:,44]
q_full[:,36] = (inpainted_data[:,45]+inpainted_data[:,46])/2
q_full[:,37] = -(inpainted_data[:,47]+inpainted_data[:,48]+inpainted_data[:,49])/3
q_full[:,39] = -(inpainted_data[:,50]+inpainted_data[:,51]+inpainted_data[:,52])/3
q_full[:,41] = -(inpainted_data[:,53]+inpainted_data[:,54]+inpainted_data[:,55])/3
q_full[:,43] = -(inpainted_data[:,56]+inpainted_data[:,57]+inpainted_data[:,58])/3

q_full[:,38] = 1.05851325*q_full[:,37]+0.72349796
q_full[:,40] = 1.05851325*q_full[:,39]+0.72349796
q_full[:,42] = 1.05851325*q_full[:,41]+0.72349796
q_full[:,44] = 1.05851325*q_full[:,43]+0.72349796

q_robot = np.zeros((L,32))
q_robot[:,:5] = q_full[:,30:35]
q_robot[:,5] = np.radians(30)
q_robot[:,6:14] = q_full[:,37:45]
q_robot[:,14:16] = q_full[:,35:37]
q_robot[:,16:21] = q_full[:,14:19]
q_robot[:,22:30] = q_full[:,21:29]
q_robot[:,30:32] = q_full[:,19:21]

np.save("../data/" + data_number[:2] + "a_robot.npy", q_robot)
header_name = 'robot1/joint1,robot1/joint2,robot1/joint3,robot1/joint4,robot1/joint5,robot1/joint6, \
               robot1/index_q1,robot1/index_q2,robot1/middle_q1,robot1/middle_q2,robot1/ring_q1,robot1/ring_q2,robot1/pinky_q1,robot1/pinky_q2,robot1/thumb_q1,robot1/thumb_q2, \
               robot2/joint1,robot2/joint2,robot2/joint3,robot2/joint4,robot2/joint5,robot2/joint6, \
               robot2/index_q1,robot2/index_q2,robot2/middle_q1,robot2/middle_q2,robot2/ring_q1,robot2/ring_q2,robot2/pinky_q1,robot2/pinky_q2,robot2/thumb_q1,robot2/thumb_q2'
np.savetxt("../../EMBC-hand-motion-retargeting/matlab_hand-motion-retargeting/data/motions/" + data_number[:2] + "a_robot.csv", q_robot, delimiter=',', header=header_name, fmt='%.6f')

### Visualization

In [6]:
import os
import cv2

SAVE_VIDEO = False

video = []
tick = 0

# Initialize MuJoCo viewer
env.init_viewer(viewer_title='Common Rig',viewer_width=400,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,1.2])
env.reset()

while tick < L:
    q = q_full[tick,:]
    p_root = data['root'][tick,:]
    
    env.forward(q=q,INCREASE_TICK=True)
    env.set_p_root(root_name='base',p=p_root)

    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        env.render()  
        frame = env.grab_image()
        video.append(frame)

        for rev_joint_idx,rev_joint_name in zip(env.rev_joint_idxs,env.rev_joint_names):
            axis_joint = env.model.jnt_axis[rev_joint_idx]
            p_joint,R_joint = env.get_pR_joint(joint_name=rev_joint_name)
            axis_world = R_joint@axis_joint
            axis_rgba = np.append(np.eye(3)[:,np.argmax(axis_joint)],0.2)
            axis_len,axis_r = 0.02,0.002
            env.plot_arrow_fr2to(
                p_fr=p_joint,p_to=p_joint+axis_len*axis_world,
                r=axis_r,rgba=axis_rgba)

    tick = tick + 1
    if tick == L:
        if not SAVE_VIDEO: tick = 0

env.close_viewer()

if SAVE_VIDEO:
    shape = video[0].shape
    video_folder = ("../video/")
    if not os.path.isdir(video_folder): os.mkdir(video_folder)

    video_path = os.path.join(video_folder, "robot" + data_number[:2] + ".mp4")
    video_fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
    fps = 50
    video_out = cv2.VideoWriter(video_path, video_fourcc, fps, (shape[1], shape[0]), True)

    for i in range(len(video)):
        video_frame = cv2.cvtColor(video[i], cv2.COLOR_BGR2RGB)
        video_out.write(video_frame)
    video_out.release()

Pressed ESC
Quitting.


Exception: GLFW window does not exist but you tried to render.